# Data 516 Homework 1
Author: Ashwin Naresh

In this assignment, we will be cleaning and validating server quote data and using it to create some tables. 

We need to perform the following validations on our quote data:

Quotes for a given month can only be accepted starting on the first Monday of the month and ending on the 25th. These boundary dates are inclusive. Any quotes provided outside of these dates will not be considered. The sum of the quote line details must match the provided quote summary total. If the two datasets do not agree, the quotes will not be considered. We do not purchase Program C and Program E quotes from Vendor 7. Vendor 7’s systems are configured to quote all available programs, so these quotes need to be discarded.

We will create the following tables:
- The table of extended quantites per component for all programs.
- The table of total cost per program per vendor, based on the latest received quote.
- The table of total cost per program per vendor, based on the first received quote.
- The table of "best-in-class" total cost per program (regardless of vendor).

In [1]:
try:
    from datetime import datetime
    import polars as pl
    import pandas as pd
    import openpyxl
except ImportError:
    !pip install polars
    import polars as pl

## Data Cleaning and Validation
In this step, we will read in the various data files and clean/validate the data according to our specifications. We will be joining these data sets together in order to create the tables in the next step.

In [2]:
df_quotelines_raw = pl.read_csv("quote_lines.csv")

# Remove entries that are not between the first monday and the 25th (Sept 2nd through Sept 25th)
df_quotelines_timefilter = (
    df_quotelines_raw
    .with_columns(pl.col("quote_timestamp").str.to_datetime())
    .filter(pl.col("quote_timestamp").is_between(datetime(2024, 9, 2), datetime(2024, 9, 26)))
)

# Remove program C and program E quotes from Vendor 7
df_quotelines_noCE = (
    df_quotelines_timefilter
    .filter((pl.col("Vendor") != "Vendor_7") | ((pl.col("Program") != "Program_C") & (pl.col("Program") != "Program_E")))
)

In [3]:
# This function reads in the server config and rack config files, transposes them, and clears up the table
def read_config_and_adjust(df):
    df_transposed = df.transpose(include_header=True)
    df_transposed = df_transposed.rename({"column": "Program"})
    columns = ["Program"] + df["Item"].to_list()
    df_transposed.columns = columns
    df_dropped = df_transposed.slice(1)
    return df_dropped

# Read in the configuration files
df_server_raw = pl.read_csv("server_configurations.csv")
df_rack_raw = pl.read_csv("rack_configurations.csv")

df_server_specs = read_config_and_adjust(df_server_raw)
df_rack_specs = read_config_and_adjust(df_rack_raw)

# Join the server and rack config tables
df_joined = df_server_specs.join(df_rack_specs, on="Program", how="inner")

# Cast the numerical fields to floats and rename the columns
df_config = df_joined.with_columns([
    pl.col("CPU").cast(pl.Float64).alias("CPU_COUNT"),
    pl.col("GPU").cast(pl.Float64).alias("GPU_COUNT"),
    pl.col("RAM").cast(pl.Float64).alias("RAM_COUNT"),
    pl.col("SSD").cast(pl.Float64).alias("SSD_COUNT"),
    pl.col("HDD").cast(pl.Float64).alias("HDD_COUNT"),
    pl.col("MOBO").cast(pl.Float64).alias("MOBO_COUNT"),
    pl.col("NIC").cast(pl.Float64).alias("NIC_COUNT"),
    pl.col("PSU").cast(pl.Float64).alias("PSU_COUNT"),
    pl.col("TRAY").cast(pl.Float64).alias("TRAY_COUNT"),
    pl.col("SERVERS").cast(pl.Float64).alias("SERVER_COUNT"),
    pl.col("TOR").cast(pl.Float64).alias("TOR_COUNT"),
    pl.col("CHASSIS").cast(pl.Float64).alias("CHASSIS_COUNT")
]).drop("CPU","GPU","RAM","SSD","HDD","MOBO","NIC","PSU","TRAY","SERVERS","TOR","CHASSIS")

# Rename the entries in the Program column to make the next merge easier
df_config_final = df_config.with_columns(
    pl.Series(["Program_A", "Program_B", "Program_C", "Program_D", "Program_E"]).alias("Program")
)


In [4]:
# Join the quote data with the config data
df_quote_config = df_quotelines_noCE.join(df_config_final, on="Program", how="inner")

# Calculate the totals
df_quote_config_totals = df_quote_config.with_columns(
    ((pl.col("SERVER_COUNT") * 
    ((pl.col("CPU") * pl.col("CPU_COUNT")) + 
    (pl.col("GPU") * pl.col("GPU_COUNT")) +
    (pl.col("RAM") * pl.col("RAM_COUNT")) +
    (pl.col("SSD") * pl.col("SSD_COUNT")) +
    (pl.col("HDD") * pl.col("HDD_COUNT")) +
    (pl.col("MOBO") * pl.col("MOBO_COUNT")) +
    (pl.col("NIC") * pl.col("NIC_COUNT")) +
    (pl.col("PSU") * pl.col("PSU_COUNT")) +
    (pl.col("TRAY") * pl.col("TRAY_COUNT")))) + 
    (pl.col("TOR") * pl.col("TOR_COUNT")) + 
    (pl.col("CHASSIS") * pl.col("CHASSIS_COUNT"))).alias("TOTAL")
)

# Read in the quote summary data get it ready for joining
df_quotesummary_raw = pl.read_csv("quote_summaries.csv")
df_quotesummary_dt = (
    df_quotesummary_raw
    .with_columns(pl.col("quote_timestamp").str.to_datetime())
    .with_columns(pl.concat_str([pl.lit("Program_"), pl.col("program")]).alias("program"))
)

# Join the quote config data with the summary data and drop the rows where the totals do not match
df_cleaned = (
    df_quote_config_totals
    .join(
        df_quotesummary_dt,
        left_on=["Vendor", "Program", "quote_timestamp"],
        right_on=["vendor", "program", "quote_timestamp"],
        how="inner"
    )
    .with_columns(pl.col("TOTAL").round(2))
    .filter(pl.col("TOTAL") == pl.col("reported_total_price"))
    .drop(pl.col("reported_total_price"))
)

df_cleaned.write_csv("output.csv")

## Table Creation
In this step, we will be creating our output tables. Each of the following code cell contains the code for creating one of the four tables.

In [5]:
# Table 1: The table of extended quantites per component for all programs.
# Each row represents a distinct component and each attribute in the row corresponds to a unique program.

# Drop the other columns
df_t1_1 = df_cleaned.drop("CPU", "GPU", "RAM", "SSD", "HDD", "MOBO", "NIC", "PSU", "TRAY", "TOR", "CHASSIS", "quote_timestamp", "Vendor", "TOTAL")

# Caluclate the total counts
columns = ["CPU_COUNT", "GPU_COUNT", "RAM_COUNT", "SSD_COUNT", "HDD_COUNT", "MOBO_COUNT", "NIC_COUNT", "PSU_COUNT", "TRAY_COUNT"]
df_t1_2 = (df_t1_1
    .with_columns((pl.col(col) * pl.col("SERVER_COUNT")).alias(col) for col in columns)
    .drop("SERVER_COUNT")
)

# Aggreagate the counts by Program
df_t1_3 = df_t1_2.group_by("Program").agg([
    pl.sum(col).alias(col) for col in df_t1_2.columns[1:]
])

# Unpivot then pivot to transpose the table
df_t1_4 = df_t1_3.unpivot(index="Program", variable_name="Component")
df_t1_4 = df_t1_4.pivot(on="Program", index="Component", values="value", aggregate_function="sum", sort_columns=True)

# Convert to pandas dataframe for better display
df_pandas = df_t1_4.to_pandas()
print(df_pandas)

        Component  Program_A  Program_B  Program_C  Program_D  Program_E
0       CPU_COUNT      648.0      336.0      868.0      320.0      320.0
1       GPU_COUNT        0.0      672.0        0.0        0.0      640.0
2       RAM_COUNT     1296.0      672.0     1736.0     2560.0     2560.0
3       SSD_COUNT      324.0      336.0      434.0      320.0        0.0
4       HDD_COUNT        0.0        0.0        0.0     6400.0     6400.0
5      MOBO_COUNT      324.0      168.0      434.0      320.0      320.0
6       NIC_COUNT      648.0      336.0      868.0      640.0      320.0
7       PSU_COUNT      324.0      336.0      434.0      320.0      320.0
8      TRAY_COUNT      324.0      168.0      434.0      320.0      320.0
9       TOR_COUNT       27.0       42.0       31.0       32.0       32.0
10  CHASSIS_COUNT       27.0       21.0       31.0       32.0       32.0


In [6]:
# Table 2: The table of total cost per program per vendor, based on the latest received quote.

df_t2_1 = df_cleaned.select(["Program", "Vendor", "quote_timestamp", "TOTAL"])
sorted_df = df_t2_1.sort("quote_timestamp", descending=True)

df_t2_2 = sorted_df.group_by(["Program", "Vendor"]).agg(pl.all().first())
df_t2_3 = df_t2_2.sort(["Program", "Vendor"]).drop("quote_timestamp")

# Convert to pandas dataframe for better display
df_pandas = df_t2_3.to_pandas()
print(df_pandas)

      Program    Vendor     TOTAL
0   Program_A  Vendor_1  18226.44
1   Program_A  Vendor_2  17658.75
2   Program_A  Vendor_3  18337.45
3   Program_A  Vendor_4  18097.66
4   Program_A  Vendor_5  17566.51
5   Program_A  Vendor_6  17916.74
6   Program_A  Vendor_7  18200.79
7   Program_B  Vendor_1  31664.18
8   Program_B  Vendor_2  31166.21
9   Program_B  Vendor_3  32049.37
10  Program_B  Vendor_4  29267.06
11  Program_B  Vendor_5  29148.99
12  Program_B  Vendor_6  28146.80
13  Program_B  Vendor_7  31640.69
14  Program_C  Vendor_1  20237.50
15  Program_C  Vendor_2  20375.36
16  Program_C  Vendor_3  20095.09
17  Program_C  Vendor_4  20938.11
18  Program_C  Vendor_5  21211.93
19  Program_C  Vendor_6  21211.09
20  Program_D  Vendor_1  38139.87
21  Program_D  Vendor_2  50639.78
22  Program_D  Vendor_3  48570.62
23  Program_D  Vendor_4  41156.46
24  Program_D  Vendor_5  52326.86
25  Program_D  Vendor_6  47118.18
26  Program_D  Vendor_7  34309.11
27  Program_E  Vendor_1  56903.34
28  Program_E 

In [7]:
# Table 3: The table of total cost per program per vendor, based on the first received quote.

df_t3_1 = df_cleaned.select(["Program", "Vendor", "quote_timestamp", "TOTAL"])
sorted_df = df_t3_1.sort("quote_timestamp", descending=False)

df_t3_2 = sorted_df.group_by(["Program", "Vendor"]).agg(pl.all().first())
df_t3_3 = df_t3_2.sort(["Program", "Vendor"]).drop("quote_timestamp")

# Convert to pandas dataframe for better display
df_pandas = df_t3_3.to_pandas()
print(df_pandas)

      Program    Vendor     TOTAL
0   Program_A  Vendor_1  17555.80
1   Program_A  Vendor_2  18287.76
2   Program_A  Vendor_3  18934.65
3   Program_A  Vendor_4  18547.48
4   Program_A  Vendor_5  17768.43
5   Program_A  Vendor_6  17866.88
6   Program_A  Vendor_7  18099.99
7   Program_B  Vendor_1  30304.36
8   Program_B  Vendor_2  30943.99
9   Program_B  Vendor_3  30001.48
10  Program_B  Vendor_4  29267.06
11  Program_B  Vendor_5  30455.55
12  Program_B  Vendor_6  32021.13
13  Program_B  Vendor_7  31640.69
14  Program_C  Vendor_1  21839.65
15  Program_C  Vendor_2  21206.47
16  Program_C  Vendor_3  21841.24
17  Program_C  Vendor_4  20741.65
18  Program_C  Vendor_5  20617.44
19  Program_C  Vendor_6  20719.93
20  Program_D  Vendor_1  35885.62
21  Program_D  Vendor_2  45809.01
22  Program_D  Vendor_3  39012.08
23  Program_D  Vendor_4  39211.48
24  Program_D  Vendor_5  54559.00
25  Program_D  Vendor_6  44603.39
26  Program_D  Vendor_7  39209.79
27  Program_E  Vendor_1  58606.86
28  Program_E 

In [8]:
# Table 4: The table of "best-in-class" total cost per program (regardless of vendor).

df_t4_1 = df_cleaned.drop("quote_timestamp", "Vendor", "TOTAL")

df_t4_2 = df_t4_1.group_by("Program").agg([
    pl.min(col) for col in df_t4_1.columns[1:]
]).sort("Program")

df_t4_3 = df_t4_2.with_columns(
    ((pl.col("SERVER_COUNT") * 
    ((pl.col("CPU") * pl.col("CPU_COUNT")) + 
    (pl.col("GPU") * pl.col("GPU_COUNT")) +
    (pl.col("RAM") * pl.col("RAM_COUNT")) +
    (pl.col("SSD") * pl.col("SSD_COUNT")) +
    (pl.col("HDD") * pl.col("HDD_COUNT")) +
    (pl.col("MOBO") * pl.col("MOBO_COUNT")) +
    (pl.col("NIC") * pl.col("NIC_COUNT")) +
    (pl.col("PSU") * pl.col("PSU_COUNT")) +
    (pl.col("TRAY") * pl.col("TRAY_COUNT")))) + 
    (pl.col("TOR") * pl.col("TOR_COUNT")) + 
    (pl.col("CHASSIS") * pl.col("CHASSIS_COUNT"))).alias("TOTAL")
)

df_t4_4 = df_t4_3.select(["Program", "CPU", "GPU", "RAM", "SSD", "HDD", "MOBO", "NIC", "PSU", "TRAY", "TOR", "CHASSIS", "TOTAL"])

# Convert to pandas dataframe for better display
df_pandas = df_t4_4.to_pandas()
print(df_pandas)

     Program     CPU     GPU    RAM     SSD     HDD   MOBO    NIC    PSU  \
0  Program_A  300.29  400.41  44.22  180.83  105.42  81.27  20.28  81.49   
1  Program_B  300.53  403.88  44.42  180.59  102.50  81.36  20.64  82.23   
2  Program_C  302.02  400.56  44.18  180.34  103.75  81.36  20.59  82.47   
3  Program_D  301.36  408.34  44.60  180.24  100.89  80.72  20.98  81.97   
4  Program_E  301.77  410.57  44.11  180.53  101.95  81.18  20.19  80.39   

    TRAY     TOR  CHASSIS     TOTAL  
0  19.99  507.45  1097.67  15784.32  
1  12.33  503.87  1018.23  26464.93  
2  14.48  502.95  1007.48  18038.69  
3  11.57  508.13  1015.84  32248.17  
4  11.57  523.49  1009.27  38613.96  
